[Website](https://nigeriapropertycentre.com/for-sale/houses/showtype)

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
import pandas as pd
import time

In [2]:
# url = "https://nigeriapropertycentre.com/for-sale/houses/showtype?page=1"

url = "https://nigeriapropertycentre.com/for-sale/houses/detached-duplexes/lagos/lekki/ikota/2559353-five-bedroom-fully-detached-duplex"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html')
response

<Response [200]>

In [ ]:
properties = []


for page in range(1,2):
    path = "C:/Users/HP/Downloads/chromedriver-win64/chromedriver.exe"
    url = f"https://nigeriapropertycentre.com/for-sale/houses/showtype?page={page}"

    service = Service(path)
    driver = webdriver.Chrome(service=service)

    driver.get(url)
    property_links = [link.get_attribute('href') for link in driver.find_elements(By.XPATH, '//a[@itemprop="url"]')]

    for link_url in property_links:
        driver.get(link_url)

        time.sleep(3)

        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html')

        rows = soup.find('table', class_= 'table table-bordered table-striped')

        
        for row in rows:
            line = row.find_all('td')
            fields = [h.text.strip() for h in line]
            

            table = {
                field.split(":")[0].strip(): field.split(":")[1].strip() 
                for field in fields if ":" in field # and len(field.split(":")) > 1
                }

            figure = soup.find_all('span', class_='price')[1]
            price = float(figure.attrs['content'])
            table['Price'] = price

            address = soup.find('div', class_='col-sm-8 f15 property-details')
            location = address.text.strip().split(',')[-2:]
            city = location[0].strip()
            state = location[1].strip()
            table['District'] = city 
            table['State'] = state

            properties.append(table)

            # back = driver.find_element(By.XPATH, '//a[@class="underline"]')
            # back.click()
        driver.back()
        time.sleep(2) 

    next_icon = driver.find_element(By.XPATH, "//a[@rel='next']")
    next_icon.click()

# driver.quit()

In [6]:
properties

[{'Property Ref': '2338229',
  'Added On': '05 Jul 2024',
  'Last Updated': '08 Nov 2024',
  'Market Status': 'Available',
  'Type': 'Semi-detached Duplex',
  'Bedrooms': '5',
  'Bathrooms': '6',
  'Toilets': '6',
  'Parking Spaces': '5',
  'Total Area': '600 sqm',
  'Covered Area': '600 sqm',
  'Price': 600000000.0,
  'District': '5 bedroom semi-detached duplex for sale\n \xa0Asokoro District',
  'State': 'Abuja'},
 {'Property Ref': '2515491',
  'Added On': '15 Oct 2024',
  'Last Updated': '18 Nov 2024',
  'Market Status': 'Available',
  'Type': 'Detached Duplex',
  'Bedrooms': '5',
  'Bathrooms': '5',
  'Toilets': '6',
  'Parking Spaces': '6',
  'Price': 320000000.0,
  'District': 'Lekki',
  'State': 'Lagos'},
 {'Property Ref': '2570105',
  'Added On': '14 Nov 2024',
  'Last Updated': '18 Nov 2024',
  'Market Status': 'Available',
  'Type': 'Detached Duplex',
  'Bedrooms': '5',
  'Bathrooms': '5',
  'Toilets': '6',
  'Parking Spaces': '10',
  'Servicing': 'Serviced',
  'Price': 35000